In [14]:
# Latex match Grobid

import os
import random

# fetch data
tex_root_path = "/home/jihuawei/jhw_pdf_parser/grobid_base/datasetgenius/formula/tmp/tex"
tex_folds_rand_sample = random.sample(os.listdir(tex_root_path), 1)


xmls = [os.path.join("/home/jihuawei/jhw_pdf_parser/grobid_base/datasetgenius/formula/tmp/xmls", f"{i}.tei.xml") for i in tex_folds_rand_sample]
# latex = [[os.path.join(f"/home/jihuawei/jhw_pdf_parser/grobid_base/datasetgenius/formula/tmp/tex/{i}", j) for j in os.listdir(f"/home/jihuawei/jhw_pdf_parser/grobid_base/datasetgenius/formula/tmp/tex/{i}")]for i in tex_folds_rand_sample]


In [15]:
tex_folds_rand_sample

['2306.09649']

In [16]:
# 提取latex种包含公式的句子
# 思路：
# 获得公式及位于的index
# 获得段落句点的index
# 看公式落在哪两个句点之间
# 提取出来
import regex as re
from collections import OrderedDict
"""
# 找出句点
(?<!No)(?<!\.g)(?<!\.e)(?<!\{)\.(?!g\.)(?!e\.)(?!\d)(?=\s)
# 提取公式
(?<!\\)(?:((?<!\$)\${1,2}(?!\$))|(\\begin\{equation\})|(\\\()|(\\\[)|(\\begin\{multline\}))(?(1)(.*?)(?<!\\)(?<!\$)\1(?!\$)|(?:([\s\S]*?)(?<!\\)(?:(?(2)\\end\{equation\}|(?(3)\\\)|(?(4)\\\]|(?(5)\\end\{multline\})))))))
# 清洗公式

"""

for i in tex_folds_rand_sample:
    for j in os.listdir(os.path.join("/home/jihuawei/jhw_pdf_parser/grobid_base/datasetgenius/formula/tmp/tex/", i)):
        with open(os.path.join("/home/jihuawei/jhw_pdf_parser/grobid_base/datasetgenius/formula/tmp/tex/", i, j), encoding='ISO-8859-1') as fp:
            data = fp.read()
        # 注释去掉
        clean_data = re.sub(r'(?<!\\)%.*\n?', '', data)
        # \begin{figure}...\end{figure}去掉
        clean_data = re.sub(r'\\begin\{figure\}[\s\S]*?\\end\{figure\}', '', clean_data)
        # \begin{algorithm}...\end{algorithm}去掉
        clean_data = re.sub(r'\\begin\{algorithm\}[\s\S]*?\\end\{algorithm\}', '', clean_data)
        # \begin{lstlisting}...\end{lstlisting}去掉
        clean_data = re.sub(r'\\begin\{lstlisting\}[\s\S]*?\\end\{lstlisting\}', '', clean_data)
        clean_data = clean_data.replace('\n', ' ')
        # 提取公式
        reg1 = r"(?<!\\)(?:((?<!\$)\${1,2}(?!\$))|(\\begin\{equation\})|(\\\()|(\\\[)|(\\begin\{multline\}))(?(1)(.*?)(?<!\\)(?<!\$)\1(?!\$)|(?:([\s\S]*?)(?<!\\)(?:(?(2)\\end\{equation\}|(?(3)\\\)|(?(4)\\\]|(?(5)\\end\{multline\})))))))"
        pattern1 = re.compile(reg1, re.MULTILINE)
        matches1 = pattern1.finditer(clean_data)   
        equ_spans = [match.span() for match in matches1]
        # 提取句点
        reg2 = r"(?<!No)(?<!\.g)(?<!\.e)(?<!\{)\.(?!g\.)(?!e\.)(?!\d)(?=\s)"
        pattern2 = re.compile(reg2, re.MULTILINE)
        matches2 = pattern2.finditer(clean_data)
        period_spans = [match.span() for match in matches2]

In [17]:
# 查询一段区间是否完全在另一段区间里面
def if_inside(span1, span2, strict=False):
    if strict:
        if span1[0] > span2[0] and span1[1] < span2[1]:
            return True
        else:
            return False   
    else:   
        if span1[0] >= span2[0] and span1[1] <= span2[1]:
            return True
        else:
            return False

In [18]:
# 对period_spans进行筛选，先把跨equation的span合起来
drop_per_lst = []
for equ_span in equ_spans:
    equ_right = equ_span[1]
    for period_span in period_spans:
        per_right = period_span[1]
        if per_right <= equ_right:
            if if_inside((per_right,per_right), equ_span):
                # print(clean_data[equ_span[0]:equ_span[1]])
                # print(period_span)
                # print("=================================================")
                drop_per_lst.append(period_span)
                
        else:
            break
for drop_per in drop_per_lst:
    period_spans.remove(drop_per)# 去掉这个句点就是合起来了
# print(period_spans)

In [19]:
# 把包含公式的句子找出来
sentence_spans = []
for i in range(len(period_spans)):
    if i == 0:
        sentence_spans.append((0, period_spans[i][1]))
    elif i == len(period_spans):
        equ_spans.append((period_spans[i][1], len(period_spans)))
    else:
        sentence_spans.append((period_spans[i-1][1], period_spans[i][1]))
equ_sentence_spans = []
for equ_span in equ_spans:
    for sentence_span in sentence_spans:
        if if_inside(equ_span, sentence_span):
            equ_sentence_spans.append(sentence_span)
            break
equ_sentence_spans = list(OrderedDict.fromkeys(equ_sentence_spans).keys())
equ_sentence_spans = [equ_sentence for equ_sentence in equ_sentence_spans if equ_sentence[1] - equ_sentence[0]<= 800]# 去掉极长的

In [20]:
def clean_equ_sentences(equ_sentence_spans, clean_data):
    equ_sentence_lst = []
    for equ_sentence_span in equ_sentence_spans:
        equ_sentence = clean_data[equ_sentence_span[0]:equ_sentence_span[1]].replace('\n',' ').strip()# 初步清洗
        equ_sentence = re.sub(r'\s+', ' ', equ_sentence)
        equ_sentence = re.sub(r'(?!\\begin\{equation\})(?!\\end\{equation\})(?!\\begin\{multline\})(?!\\end\{multline\})((\\begin\{.*?\})|(\\end\{.*?\})|(\\label\{.*?\})|\\item|\\qqq|\\ee|\\ref{.*?}|\\eqref{.*?}|\\footnote{.*?}|\\cite{.*?})', '', equ_sentence)
        equ_sentence = equ_sentence.replace('\n',' ').strip()# again
        equ_sentence = re.sub(r'\s+', ' ', equ_sentence)
        equ_sentence_lst.append(equ_sentence)
    return equ_sentence_lst
equ_sentence_lst = clean_equ_sentences(equ_sentence_spans, clean_data)


In [21]:
for i in equ_sentence_lst:
    print(i)
    print("==================================================")

We found in a user study ($N$=16) that the app generated with \mr{} significantly reduced cognitive load and task completion time compared to a GUI-only app.
The \mr{} framework ships with one \texttt{Helper class} called \texttt{DateTime} and a set of \texttt{GenieComponent} that can be used to represent the \texttt{DateTime} instance, like one that shows the data in a convenient ``$n$ seconds/minutes/hours/days ago'' format.
We used a paired t-test and found that the difference is statistically significant ($p=0.0004, t=3.955$).
We used a Wilcoxon test and found that the difference is statistically significant ($p=0.013, z=21$).
We used a Wilcoxon test and found that the difference is statistically significant ($p=0.031, z=22$).


In [22]:
from grobid_parser import GParser
import xml.etree.ElementTree as ET

clean_data = ""
pdf_parser = GParser(parse_imgs=False)
for xml_path in xmls:
    pdf_name = os.path.basename(xml_path).replace(".tei.xml", ".pdf")
    pdf_parser.pdf_name = pdf_name
    tree = ET.parse(xml_path)
    root = tree.getroot()
    result = pdf_parser.parse_all(root, pdf_name, f"{pdf_name.strip('.pdf')}.json")
    result.pop('back')
    for name in result.keys():
        top_el = result[name]
        for el in top_el:
            if 'tail' not in el.keys():
                clean_data = clean_data + el['txt'] + ' '
            else:
                clean_data = clean_data + el['txt'] + ' '
                clean_data = clean_data + el['tail'] + ' '
clean_data= clean_data.replace('\n', ' ')

In [23]:
def merge_span(span1, span2):
    return (min(span1[0], span1[1], span2[0], span2[1]), max(span1[0], span1[1], span2[0], span2[1]))

# 提取句点
reg2 = r"(?<!No)(?<!\.g)(?<!\.e)(?<!\{)\.(?!g\.)(?!e\.)(?!\d)(?=\s)"
pattern2 = re.compile(reg2, re.MULTILINE)
matches2 = pattern2.finditer(clean_data)
period_spans = [match.span() for match in matches2]
# 把包含公式的句子找出来
sentence_spans = []
for i in range(len(period_spans)):
    if i == 0:
        sentence_spans.append((0, period_spans[i][1]))
    elif i == len(period_spans):
        equ_spans.append((period_spans[i][1], len(period_spans)))
    else:
        sentence_spans.append((period_spans[i-1][1], period_spans[i][1]))

sentence_spans_clean = []
for i in range(len(sentence_spans)):
    if i == len(sentence_spans)-1:
        sentence_spans_clean.append(sentence_spans[i])
    else:
        if sentence_spans[i][1]-sentence_spans[i][0] <= 12:
            sentence_spans[i+1] = merge_span(sentence_spans[i], sentence_spans[i+1])
        else:
            sentence_spans_clean.append(sentence_spans[i])
sentence_spans = [sentence_span for sentence_span in sentence_spans_clean if sentence_span[1]-sentence_span[0] <= 800]
xml_sentence_lst = clean_equ_sentences(sentence_spans, clean_data)


In [24]:
for i in xml_sentence_lst:
    print(i)
    print("=================================================================")

ReactGenie: An Object-Oriented State Abstraction for Complex Multimodal Interactions Using Large Language Models In this paper, we present ReactGenie, a programming framework that uses a shared object-oriented state abstraction to support building complex multimodal mobile applications.
Having different modalities share the same state abstraction allows developers using ReactGenie to seamlessly integrate and compose these modalities to deliver multimodal interaction.
ReactGenie is a natural extension to the existing workflow of building a graphical app, like the workflow with React-Redux.
Developers only have to add a few annotations and examples to indicate how natural language is mapped to the user-accessible functions in 1 INTRODUCTION Multimodal interactions allow users to use a combination of input and output modalities, such as touch, voice, and graphical user interfaces (GUIs).
They have been shown to offer more flexibility, efficiency, and adaptability for diverse users and tas

In [25]:
from thefuzz import fuzz
from thefuzz import process

matched_pairs = []
choices = equ_sentence_lst
for xml_sentence in xml_sentence_lst:
    matched = process.extractOne(xml_sentence, choices, scorer=fuzz.partial_ratio)
    equ_text = matched[0]
    score = matched[1]
    if score >= 90 and len(xml_sentence)/len(equ_text) >= 0.85 and len(xml_sentence)/len(equ_text) <= 1.18:
        matched_pairs.append((xml_sentence, equ_text))

In [26]:
for i in matched_pairs:
    print(i[0])
    print(i[1])
    print("==================================")

-We found in a user study ( =16) that the app generated with ReactGenie significantly reduced cognitive load and task completion time compared to a GUI-only app.
We found in a user study ($N$=16) that the app generated with \mr{} significantly reduced cognitive load and task completion time compared to a GUI-only app.
We used a paired t-test and found that the difference is statistically significant ( = 0.0004, = 3.955).
We used a paired t-test and found that the difference is statistically significant ($p=0.0004, t=3.955$).
We used a Wilcoxon test and found that the difference is statistically significant ( = 0.013, = 21).
We used a Wilcoxon test and found that the difference is statistically significant ($p=0.013, z=21$).
We used a Wilcoxon test and found that the difference is statistically significant ( = 0.031, = 22).
We used a Wilcoxon test and found that the difference is statistically significant ($p=0.031, z=22$).


In [27]:
len()/len("")

0.30268199233716475

In [42]:
a = r"Using the fact that Z in Eq.( [1] ) is close to 1, it is possible to approximate the occupation probability of the binding site by [16] : P ij = 1 1 + 1/(n j W ij ) .In the following, we denote by n the maximum number of TF molecules that can arise when a gene is fully on, taking for simplicity "
b = r"Using the fact that $Z$ in Eq.~() is close to 1, it is possible to approximate the occupation probability of the binding site by~: \begin{equation} P_{ij} = \frac{1}{1 + 1/(n_j W_{ij})} \; . \end{equation} In the following, we denote by $n$ the maximum number of TF molecules that can arise when a gene is fully ``on'', taking for simplicity this maximum to be independent of $j$."
fuzz.ratio(a,b)

83